In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.calibration import calibration_curve
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pickle as pkl
import pandas as pd
dfs=dict()
for r in range (16,21):
    with open(f"../data/df/0.15/df_20{r}_0.15.pkl", "rb") as f:
        file = pkl.load(f)
        dfs[r] = pd.DataFrame(file)

In [ ]:
dfs[16].loc[dfs[16].Game_id==1].Goal.unique()

In [ ]:
df=pd.concat([dfs[r] for r in range (16,20)]) 
df.head(5)

In [ ]:
df["Game_id"].nunique(),df.shape[0]

In [ ]:
label_encoder = LabelEncoder()
df['Goal'] = label_encoder.fit_transform(df['Goal'])

In [ ]:
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(df['Shot_distance'], df['Goal'], test_size=0.2, random_state=12)

X_train = X_train.values.reshape(-1, 1)
X_valid = X_valid.values.reshape(-1, 1)

# Train a Logistic Regression model on the distance feature
clf_distance = LogisticRegression()
clf_distance.fit(X_train, y_train)

# Make predictions on the validation set
y_pred_distance = clf_distance.predict(X_valid)

# Calculate and print the accuracy
accuracy_distance = accuracy_score(y_valid, y_pred_distance)
print(f'Accuracy with distance feature: {accuracy_distance}')


In [ ]:
# Perform label encoding on the 'Goal' column
label_encoder = LabelEncoder()
df['Goal'] = label_encoder.fit_transform(df['Goal'])

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(df[['Shot_distance']], df['Goal'], test_size=0.2, random_state=42)

# Train a Logistic Regression model on the distance feature
clf_distance = LogisticRegression()
clf_distance.fit(X_train, y_train)

# Get predicted probabilities for the validation set
y_prob_distance = clf_distance.predict_proba(X_valid)[:, 1]

def plots(y_valid, y_prob, model_name):
    # Create a single figure with two subplots in one row
    #fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    plt.suptitle(f'Performance Evaluation of: {model_name}', fontsize=16)


    # Plot ROC curve and calculate AUC
    fpr, tpr, thresholds = roc_curve(y_valid, y_prob)
    roc_auc = roc_auc_score(y_valid, y_prob)

    axes[0,0].plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    axes[0,0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0,0].set_xlim([0.0, 1.0])
    axes[0,0].set_ylim([0.0, 1.05])
    axes[0,0].set_xlabel('False Positive Rate')
    axes[0,0].set_ylabel('True Positive Rate')
    axes[0,0].set_title('Receiver Operating Characteristic (ROC)')
    axes[0,0].legend(loc="lower right")

    

    axes[0,1].plot([0, 1], [0, 1], 'k:', label="Perfectly calibrated")
    prob_true, prob_pred = calibration_curve(y_valid, y_prob, n_bins=10)
    axes[0,1].plot(prob_pred, prob_true, 's-', label="%s" % ('clf_distance',))
    axes[0,1].set_ylabel('Fraction of positives')
    axes[0,1].set_xlabel('Mean predicted probability')
    axes[0,1].set_title('Calibration Plot')




    # Calculate and plot the rate of goals and cumulative proportion of goals
    sorted_indices = np.argsort(y_prob)
    sorted_goals = y_valid.iloc[sorted_indices]
    cumulative_goals = np.cumsum(sorted_goals)
    cumulative_total = np.arange(1, len(y_valid) + 1)

    rate_of_goals = cumulative_goals / cumulative_total

    axes[1,0].plot(np.arange(1, len(y_valid) + 1), rate_of_goals)
    axes[1,0].set_xlabel('Centile of Probability')
    axes[1,0].set_ylabel('Rate of Goals')
    axes[1,0].set_title('Rate of Goals vs. Centile of Probability')

    axes[1,1].plot(np.arange(1, len(y_valid) + 1), cumulative_goals / max(cumulative_goals))
    axes[1,1].set_xlabel('Centile of Probability')
    axes[1,1].set_ylabel('Cumulative Proportion of Goals')
    axes[1,1].set_title('Cumulative Proportion of Goals vs. Centile of Probability')

    # Display the figure with two subplots side by side
    plt.show()


plots(y_valid,y_prob_distance,"LR using Shot Distance")

In [ ]:

def plots(y_valid, y_prob, model_name, exp=None):
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    plt.suptitle(f'Performance Evaluation of: {model_name}', fontsize=16)


    # Plot ROC curve and calculate AUC
    fpr, tpr, thresholds = roc_curve(y_valid, y_prob)
    roc_auc = roc_auc_score(y_valid, y_prob)

    axes[0,0].plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    axes[0,0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0,0].set_xlim([0.0, 1.0])
    axes[0,0].set_ylim([0.0, 1.05])
    axes[0,0].set_xlabel('False Positive Rate')
    axes[0,0].set_ylabel('True Positive Rate')
    axes[0,0].set_title('Receiver Operating Characteristic (ROC)')
    axes[0,0].legend(loc="lower right")

    

    axes[0,1].plot([0, 1], [0, 1], 'k:', label="Perfectly calibrated")
    prob_true, prob_pred = calibration_curve(y_valid, y_prob, n_bins=10)
    axes[0,1].plot(prob_pred, prob_true, 's-', label="%s" % ('clf_distance',))
    axes[0,1].set_ylabel('Fraction of positives')
    axes[0,1].set_xlabel('Mean predicted probability')
    axes[0,1].set_title('Calibration Plot')

    # Calculate and plot the rate of goals and cumulative proportion of goals
    sorted_indices = np.argsort(y_prob)
    sorted_goals = y_valid[sorted_indices]
    predicted_probs = y_prob[sorted_indices]

    n_bins = 20
    bins = np.linspace(0, 1, n_bins + 1)
    midpoints = (bins[:-1] + bins[1:]) / 2

    goal_rates = []
    for i in range(n_bins):
        start_idx = int(i * len(predicted_probs) / n_bins)
        end_idx = int((i + 1) * len(predicted_probs) / n_bins)
        
        goals = sum(sorted_goals[start_idx:end_idx])
        total_shots = end_idx - start_idx
        
        goal_rate = 100 * goals / total_shots
        goal_rates.append(goal_rate)

    axes[1,0].plot(midpoints*100, goal_rates, linestyle='-')
    axes[1,0].set_xlim([100,0])
    axes[1,0].set_ylim([0,100])
    axes[1,0].set_xlabel('Centile of Probability')
    axes[1,0].set_ylabel('Rate of Goals')
    axes[1,0].set_title('Rate of Goals vs. Centile of Probability')

    cumulative_goals = np.cumsum(sorted_goals[::-1])
    tot_goals = y_valid.sum()

    axes[1,1].plot(np.arange(len(y_prob), 0, -1) * 100 / len(y_prob), cumulative_goals * 100 / tot_goals, linestyle='-')
    axes[1,1].set_xlim([100,0])
    axes[1,1].set_ylim([0,100])
    axes[1,1].set_xlabel('Centile of Probability')
    axes[1,1].set_ylabel('Cumulative Proportion of Goals')
    axes[1,1].set_title('Cumulative Proportion of Goals vs. Centile of Probability')

    if exp:
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        exp.log_image(buf, name='performance.png')
        buf.close()
    else:
        plt.show()

In [ ]:
# Perform label encoding on the 'Goal' column
label_encoder = LabelEncoder()
df['Goal'] = label_encoder.fit_transform(df['Goal'])

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(df.drop(['Goal'],inplace=False,axis=1), df['Goal'], test_size=0.2, random_state=42)

# Train a Logistic Regression model on the angle feature
clf_angle = LogisticRegression()
clf_angle.fit(X_train[['Shot_angle']].abs(), y_train)

# Get predicted probabilities for the validation set
y_prob_angle = clf_angle.predict_proba(X_valid[['Shot_angle']])[:, 1]

plots(y_valid.values,y_prob_angle,"LR using Shot Angle")

# Train a Logistic Regression model on both distance and angle features
X_train_both = X_train[['Shot_distance', 'Shot_angle']].abs()
X_valid_both = X_valid[['Shot_distance', 'Shot_angle']].abs()
clf_both = LogisticRegression()
clf_both.fit(X_train_both, y_train)

# Get predicted probabilities for the validation set
y_prob_both = clf_both.predict_proba(X_valid_both)[:, 1]

plots(y_valid.values,y_prob_both,"LR using Shot Distance and Shot Angle")


* Game_id: A unique identifier for each hockey game, allowing you to associate the data with a specific match.
* Game_time: The time elapsed within the game when the event occurred, typically in minutes and seconds.
* Period: Indicates which period of the game the event took place in (e.g., 1st period, 2nd period, etc.).
* Time: The time elapsed within the current period when the event occurred, usually in minutes and seconds.
* Team: The name or identifier of the team that performed the event.
* OppTeam: The name or identifier of the opposing team.
* Goal: A binary variable indicating whether a goal was scored (1 for a goal, 0 for no goal).
* X: The x-coordinate on the ice rink where the event occurred. This provides spatial information.
* Y: The y-coordinate on the ice rink where the event occurred. This, too, provides spatial information.
* Shooter: The player who took the shot or performed the event.
* Goalie: The goalkeeper (goalie) who was defending the net during the event.
* Type: Describes the type of event, which could include actions like a shot, a pass, a hit, or a faceoff, among others.
* Empty_net: A binary variable indicating whether the opposing team's net was empty (1 for empty, 0 for not empty). This is relevant in situations where the opposing team pulls their goalie to add an extra skater.
* Strength: Describes the on-ice strength situation, such as even strength (5 players vs. 5 players), power play (one team has more players due to a penalty), or penalty kill (one team has fewer players due to a penalty).
* Previous Event_type: The type of the previous event that occurred in the game.
* Previous_x: The x-coordinate of the previous event on the ice rink.
* Previous_y: The y-coordinate of the previous event on the ice rink.
* Previous Time Since: The time elapsed since the previous event occurred.
* Previous Distance: The distance between the current event and the previous event on the ice rink.
* Speed: The speed or velocity associated with the event, typically calculated based on the change in position and time.
* Is Rebound: A binary variable indicating whether the event was a rebound (1 for rebound, 0 for not a rebound).
* X_net: The x-coordinate of the net where the event occurred.
* Shot_distance: The distance between the shooter and the net when the event occurred.
* Shot_angle: The angle at which the shot was taken in relation to the net.
* Rebound Angle: The angle at which a rebound was taken in relation to the net.
* Year: The year in which the game took place or the data was recorded.